# 210

老实说，这题自己想的解法简直烂透了。

但我还是要记下来。

首先，unvisited 还是初始化为 list(range(n))，然后每次从中拿出第一个元素，并检查她「还没有满足的」先修课程，
也就是找出哪些同样在 unvisited 里面还没修的课程，但又是她依赖的课程。存在 `unsatisfied` 里面。

如果发现没有未完成的依赖，那么简单了。直接把它加入 sequence 里面，然后从 unvisited 里面删掉它。

如果发现有未完成的依赖，那么就把它插入到 unvisited 列表里面的最前头，然后重新开始这个循环。

那么，如何处理死循环的情况呢？简单，如果死循环，那一定是 unvisited 列表循环出现了。
那么我们就把 列表转成 tuple（因为这样才能 hash……）然后放到一个 happened_sequence 的 set 里做检查。
一旦出现了重复情况，直接 return []。再见。

就是这么个朴素的思路…
最低限度地过了 cases…

Runtime: 1644 ms, faster than 5.01% of Python3 online submissions for Course Schedule II.
Memory Usage: 77.4 MB, less than 7.14% of Python3 online submissions for Course Schedule II.

另外的思路：
深度优先搜索法：

➔ let S be a stack of courses
➔ function dfs(node)
➔     for each neighbor in adjacency list of node
➔          dfs(neighbor)
➔     add node to S  

代码如下：

In [ ]:
from collections import defaultdict
class Solution:

    WHITE = 1
    GRAY = 2
    BLACK = 3

    def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """

        # Create the adjacency list representation of the graph
        adj_list = defaultdict(list)

        # A pair [a, b] in the input represents edge from b --> a
        for dest, src in prerequisites:
            adj_list[src].append(dest)

        topological_sorted_order = []
        is_possible = True

        # By default all vertces are WHITE
        color = {k: Solution.WHITE for k in range(numCourses)}
        def dfs(node):
            nonlocal is_possible

            # Don't recurse further if we found a cycle already
            if not is_possible:
                return

            # Start the recursion
            color[node] = Solution.GRAY

            # Traverse on neighboring vertices
            if node in adj_list:
                for neighbor in adj_list[node]:
                    if color[neighbor] == Solution.WHITE:
                        dfs(neighbor)
                    elif color[neighbor] == Solution.GRAY:
                         # An edge to a GRAY vertex represents a cycle
                        is_possible = False

            # Recursion ends. We mark it as black
            color[node] = Solution.BLACK
            topological_sorted_order.append(node)

        for vertex in range(numCourses):
            # If the node is unprocessed, then call dfs on it.
            if color[vertex] == Solution.WHITE:
                dfs(vertex)

        return topological_sorted_order[::-1] if is_possible else []

In [ ]:
# 另一个思路是先遍历整个 preseqs 表，把每个节点（就是每门课啦）的入度出度先弄个清楚。

from collections import defaultdict
class Solution:

    def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """

        # Prepare the graph
        adj_list = defaultdict(list)
        indegree = {}
        for dest, src in prerequisites:
            adj_list[src].append(dest)

            # Record each node's in-degree
            indegree[dest] = indegree.get(dest, 0) + 1

        # Queue for maintainig list of nodes that have 0 in-degree
        zero_indegree_queue = [k for k in range(numCourses) if k not in indegree]

        topological_sorted_order = []

        # Until there are nodes in the Q
        while zero_indegree_queue:

            # Pop one node with 0 in-degree
            vertex = zero_indegree_queue.pop(0)
            topological_sorted_order.append(vertex)

            # Reduce in-degree for all the neighbors
            if vertex in adj_list:
                for neighbor in adj_list[vertex]:
                    indegree[neighbor] -= 1

                    # Add neighbor to Q if in-degree becomes 0
                    if indegree[neighbor] == 0:
                        zero_indegree_queue.append(neighbor)

        return topological_sorted_order if len(topological_sorted_order) == numCourses else []
    
# 总有一门课的入度为 0，即不需要先修课程的。以他为起点，进行treebuild 会好很多。

In [ ]:
# Analysis written by: @sachinmalhotra1993.